In [1]:
from Bio import pairwise2
from Bio import SeqIO
from multiprocessing import Pool, cpu_count
import tqdm

def read_fasta(file):
    with open(file) as handle:
        return list(SeqIO.parse(handle, 'fasta')) 
    
def save_output(fasta, name):
    """
    in:
        > fasta: a list or XXX object containing BioRecords from a fasta file
        > name: a string contianing the name of the file to save the fasta to
    out:
        > A fasta file written in the pwd containing the infromation found in
          the fasta list
    """
    
    SeqIO.write(fasta, name, 'fasta')
    
def find_tat_exon_positions(seq, exon):
        """
        Scans the tat sequence for the start of the exon 1 or exon 2 sequence
        and returns its position
        """

        if exon == 1:

            # A random CARES Tat exon1 sequence used to map onto the large HIV sequence
            exon = 'atggagccggtagatcctagactagagccctggaatcatccaggaagtcagcctaagactccttg\
            taccccttgctattgtaaaaggtgttgtctgcattgccaagtttgcttcataacaaaaggcttaggcatctcctatggca\
            ggaagaagcggagaaagcgacgaagacctcctcaagacagtgaggctcatcaggaacctctatcaaagca'.upper()

            # Absurd gap penalsties to prevent any gaps from occuring
            # Also added generous match score to encourage proper mapping
            aln = pairwise2.align.localms(exon, seq, 7, 0, -50, -50)

            start_position = aln[0][0].index('A')
            end_position = start_position + 215

        elif exon == 2:

            # A random CARES Tat exon 2 sequence used to map onto the large HIV sequence
            exon = 'atccgccccccagctccgaggggacccgacaggcccggaggaatcgaagaagaaggtggagaga\
            gagacagagacagatccggtggat'.upper()

            # Absurd gap penalsties to prevent any gaps from occuring
            # Also added generous match score to encourage proper mapping
            aln = pairwise2.align.localms(exon, seq, 7, 0, -50, -50)

            start_position = aln[0][0].index('A')

            # Adds a large number of bps to end just so that we can get the longest exon 2 possible
            end_position = start_position + 200

        return start_position, end_position

def find_tat_exon(seq, exon):
    """
    Scans the Tat sequences for exon 1 or exon 2 and returns that
    portion of the seqeunce
    """

    start, end = find_tat_exon_positions(seq, exon)

    return seq[start:end]

def find_tat_cds(seq_record):
    """
    Scans the seq_record seqeunce for exons 1 and 2
    and replaces the seqeunces with the Tat CDS
    """

    exon1 = find_tat_exon(seq_record.seq, exon = 1)
    exon2 = find_tat_exon(seq_record.seq, exon = 2)

    seq_record.seq = exon1 + exon2

    return seq_record

def find_mulitple_tat_cds(seq_records):
    return [find_tat_cds(record) for record in tqdm.tqdm(seq_records)]   

In [2]:
from math import ceil

fasta = read_fasta("../Data/RawData/PacBioConsensusSeqs/combined_ungapped_P100_consensus.fasta")
no_seq_fasta = [record for record in fasta if record.seq]

# CHUNKCOUNT = 8
# chunksize = ceil(len(no_seq_fasta) / CHUNKCOUNT)

# chunks = [fasta[i:i+chunksize] for i in range(0, len(no_seq_fasta), chunksize)]
# chunks = chunks[:CHUNKCOUNT]

In [3]:
def make_full_tat(record):
    exon1 = 'atggagccggtagatcctagactagagccctggaatcatccaggaagtcagcctaagactccttg\
    taccccttgctattgtaaaaggtgttgtctgcattgccaagtttgcttcataacaaaaggcttaggcatctcctatggca\
    ggaagaagcggagaaagcgacgaagacctcctcaagacagtgaggctcatcaggaacctctatcaaagca'.upper()

    exon2 = 'atccgccccccagctccgaggggacccgacaggcccggaggaatcgaagaagaaggtggagaga\
    gagacagagacagatccggtggat'.upper()
    
    #print(seq_record)
    seq = record.seq

    # Absurd gap penalsties to prevent any gaps from occuring
    # Also added generous match score to encourage proper mapping
    exon_one_aln = pairwise2.align.localms(exon1, seq, 7, 0, -50, -50)
    exon_two_aln = pairwise2.align.localms(exon2, seq, 7, 0, -50, -50)

    start1 = exon_one_aln[0][0].index('A')
    end1 = start1 + 215

    start2 = exon_two_aln[0][0].index('A')
    end2 = start2 + 300

    big_tat = (seq[start1:end1] + seq[start2:end2]).translate()

    first_stop = big_tat.find('*')
    tat = big_tat[:first_stop]
    record.seq = tat
    
    return record
    
def make_full_tats(seq_records):
    return [make_full_tat(record) for record in tqdm.tqdm(seq_records)]

In [4]:
tats = [make_full_tat(record) for record in tqdm.tqdm(no_seq_fasta)]

i_tats = [record for record in tats if record.seq[0] == 'I']
good_tats = [record for record in tats if record.seq[0] == 'M' and len(record.seq) <= 110]

  0%|          | 0/140 [00:00<?, ?it/s]/home/bobby/.local/lib/python3.8/site-packages/Bio/Seq.py:2855: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(
100%|██████████| 140/140 [01:55<00:00,  1.21it/s]


In [5]:
#######################################################################
# Conclusion: There are some insanly long Tats that I can't just ignore
# but are too long to fit in the model. I'll keep them for safe keeping
# in case I want to revisit these later on. 
#######################################################################

for i in tats: print(f"{i.id[:9]} | {i.seq} | {len(i)}")

A0001-R09 | MEPVDPRLEPWKHPGSQPRTPCTACYCKKCCFHCQVCFTRKGLGISYGRKKRGQRRRPPQDSEAHQVSLPKQSAPQLRGDPTGPKESKKEVERETETDPVDQ | 102
A0002-R14 | MDPVDPRLEPWKHPGSQPKTPCTPCYCKKCCFHCQACFITKGLGISYGRKKRRQRRRAPPDSQTHQAPLPKQPTAQPRGDPTGPKESKKKVERETETDPVNQ | 102
A0004-R10 | MEPVDPSLEPWKHPGSQPKTACTNCYCKKCCFHCQVCFITKGLGISYGRKKRRQRRRAHQDSQIHQVPLSKQPASQPRGNPTGPKEQKKKVETEAETDQFD | 101
A0005-R05 | MDPVDPRLEPWKHPGSQPKTACTTCYCKKCCLHCQVCFITKGLGISYGRKKRRKRRRAPQDSETHQVSLSKQPTSQLRGDPTGPKESKKKVERETETDPVN | 101
A0005-R07 | MDPVDPRLEPWKHPGSQPKTACTTCYCKKCCLHCQVCFITKGLGISYGRKKRRKRRRAPQDSETHQVSLSKQPTSQLRGDPTGPKESKKKVERETETDPVN | 101
A0008-R07 | MEPVDPRLEPWKHPGSQPKTACNTCYCKRCCLHCQVCFITKGLGISYGRKKRRQRRRAAQDSETRQVPLSKQPASQRRGDPTGQKESKKKVERETETDPVH | 101
A0008-R10 | MEPVDPRLEPWKHPGSQPKTACNTCYCKKCCLHCQVCFITKGLGISYGRKKRRQRRRASQDSETRQVPLSKQSAPQRRGDPTGPKESKKKVERETETDPVH | 101
A0010-R08 | MEPVDPRLEPWKHPGSQPRTPCTPCYCKKCCFHCQVCFITKGLGISYGRKKRRQRRRPPQDSETHQASLSKQPASQPRGDPTGPKE | 86
A0010-R09 | MEPVDPRLEPWKHPGSQPKTPCTPCYCKKCCFHCQVCFITKG

In [6]:
print(f"""ORIGINAL SEQUENCE COUNTS: {len(fasta)}
PATIENT COUNTS CONTAINING SEQS: {len(no_seq_fasta)}
TAT <= 110 RESIDUES AND FIRST RESIDUE == 'M': {len(good_tats)}
---
TOTAL PATIENT COUNT: {len(set([tat.id[:5] for tat in good_tats]))}
PATIENTS:
{set([tat.id[:5] for tat in good_tats])}
---""")

for i in good_tats: print(f"{i.id[:9]} | {i.seq} | {len(i)}")

ORIGINAL SEQUENCE COUNTS: 157
PATIENT COUNTS CONTAINING SEQS: 140
TAT <= 110 RESIDUES AND FIRST RESIDUE == 'M': 133
---
TOTAL PATIENT COUNT: 97
PATIENTS:
{'A0032', 'B0147', 'A0257', 'A0408', 'A0451', 'A0004', 'A0075', 'A0439', 'A0001', 'A0040', 'A0127', 'A0380', 'B0076', 'A0447', 'B0485', 'A0037', 'A0056', 'A0310', 'A0010', 'A0325', 'A0145', 'A0143', 'A0264', 'A0036', 'A0111', 'A0008', 'A0096', 'A0241', 'A0430', 'A0433', 'A0026', 'A0107', 'A0448', 'A0133', 'A0013', 'A0086', 'A0067', 'A0038', 'A0440', 'A0342', 'A0034', 'A0501', 'A0215', 'A0025', 'A0017', 'A0002', 'A0471', 'A0095', 'A0103', 'A0278', 'B0044', 'B0522', 'A0338', 'A0132', 'A0417', 'A0005', 'A0318', 'B0146', 'A0474', 'A0091', 'A0223', 'A0239', 'A0019', 'A0357', 'A0208', 'A0359', 'A0446', 'A0449', 'A0490', 'B0303', 'A0076', 'A0500', 'A0274', 'A0083', 'A0218', 'B0037', 'B0325', 'A0045', 'A0389', 'A0068', 'A0113', 'A0109', 'A0209', 'A0059', 'A0015', 'A0213', 'A0156', 'A0255', 'A0285', 'A0401', 'A0429', 'A0379', 'A0523', 'B0538',

In [7]:
#######################################################################
# Conclusion: 'I' Tats cannot be salvaged. The rest of the
# exon just fits too cleanly for it to be a misalignment. But
# This seems interesting all things considered. Maybe it's
# A blessing in disguiase. Saving these Tats for later investigation
#######################################################################

print(f"""TOTAL TATS WHERE FIRST RESIDUE == 'I' : {len(i_tats)}
'I' TAT PATIENT COUNT: {len(set([tat.id[:5] for tat in i_tats]))}
'I' TAT PATIENTS:
{set([tat.id[:5] for tat in i_tats])}
---""")

for i in i_tats: print(f"{i.id[:9]} | {i.seq} | {len(i)}")

TOTAL TATS WHERE FIRST RESIDUE == 'I' : 2
'I' TAT PATIENT COUNT: 2
'I' TAT PATIENTS:
{'A0032', 'A0152'}
---
A0032-R05 | INPVDPSLEPWKHPGSQPKTACTKCYCKKCCFHCQVCFITKGLGISYGRKKRRQRRRAPQGSKTXQNSLPKQPTSQSRGNPTGPKESKKKVKTEAETDQFN | 101
A0152-R03 | INPVDPRLEP | 10


In [8]:
save = True

if save:
    save_output(tats, '../Data/CohortTats/PacBio/PacBio_All_Tats.fasta')
    save_output(good_tats, '../Data/CohortTats/PacBio/PacBio_Trimmed_Normal_Tats.fasta')
    save_output(i_tats, '../Data/CohortTats/PacBio/PacBio_Trimmed_I_Tats.fasta')